In [51]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

'''
Task 3: playing with NN framwork/keras and basic sentiment analysis
- use the following model as a baseline and improve it!
- export your metadata (just basic hyperparameters and outcomes for test data!)
- test data = 0.3 (not in this example, change it!)
- random_state = 4222
- no need to cross-validation!
'''

# parameters
max_fatures = 1000
embed_dim = 128
lstm_out = 196
dropout = 0.1
dropout_1d = 0.4
recurrent_dropout = 0.1
random_state = 4222
validation_size = 1000
batch_size = 16
epochs=2
verbose= 2

df = pd.read_csv('/Users/Carsten/GitRepos/NLP-LAB/Carsten_Solutions/sets/sentiment-analysis/dataset_sentiment.csv')
df = df[['text','sentiment']]
print(df[0:10])

                                                text sentiment
0  RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1  RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2  RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3  RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4  RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
5  RT @GregAbbott_TX: @TedCruz: "On my first day ...  Positive
6  RT @warriorwoman91: I liked her and was happy ...  Negative
7  Going on #MSNBC Live with @ThomasARoberts arou...   Neutral
8  Deer in the headlights RT @lizzwinstead: Ben C...  Negative
9  RT @NancyOsborne180: Last night's debate prove...  Negative


In [52]:
df = df[df.sentiment != "Neutral"]
#replace all capital letters with its small character
df['text'] = df['text'].apply(lambda x: x.lower())
#removes all rt in messages, often occuring in front of twitter raw data
df['text'] = df['text'].apply(lambda x: x.replace('rt',' '))
#only accepts alphanumerical characters, erease all other characters
df['text'] = df['text'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]','',x)))

In [53]:
#evaluate distribution of positive and negative examples
print(len(df[df.sentiment == "Positive"]))
print(len(df[df.sentiment == "Negative"]))

2236
8493


In [54]:
print(df[0:10])

                                                 text sentiment
1     scottwalker didnt catch the full gopdebate l...  Positive
3     robgeorge that carly fiorina is trending  ho...  Positive
4     danscavino gopdebate w realdonaldtrump deliv...  Positive
5     gregabbotttx tedcruz on my first day i will ...  Positive
6     warriorwoman91 i liked her and was happy whe...  Negative
8   deer in the headlights   lizzwinstead ben cars...  Negative
9     nancyosborne180 last nights debate proved it...  Negative
10  jgreendc realdonaldtrump in all fairness billc...  Negative
11    waynedupreeshow just woke up to tweet this o...  Positive
12  me reading my familys comments about how great...  Negative


In [55]:
#generates a tokenizer with fixed lenght
tok = Tokenizer(num_words=max_fatures, split=' ')
#train tokenizer
tok.fit_on_texts(df['text'].values)
#transforms each sentence to a sequence of intgers
X = tok.texts_to_sequences(df['text'].values)

#print(X[:20])

#for each of these sequences it  transforms it to an array of same length by inserting 0 in front by standard configuration of parameters 
X = pad_sequences(X)
print(X[:5])


#print(type(X))


[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0 363 122   1 722
    2  39  58 237  36 210   6 174  12 742]
 [  0   0   0   0   0   0   0   0   0   0   0  16 284 252   5 818 102 167
   26 135   6   1 172  12   2 233 723  17]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2 309  23
    1 216  12   1 702   6 185 207 371 670]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 127  17  53
  262 410   9  82 303 441  62 194   2  51]
 [  0   0   0   0   0   0   0   0   0   9 167   8  21  63   9 612 188  21
  189   4  34   1 562  19 819   2  44 743]]


In [56]:
#configuration of nn
nn = Sequential()
nn.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
nn.add(SpatialDropout1D(dropout_1d))
nn.add(LSTM(lstm_out, dropout=dropout, recurrent_dropout=recurrent_dropout))
nn.add(Dense(2, activation='softmax'))
nn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(nn.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 28, 128)           128000    
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 394       
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None


In [57]:
#get the teacher values
Y = pd.get_dummies(df['sentiment']).values
#gnerates X and y train & test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.30, random_state = random_state)
#trains the machine learning configuration with the declared parameters in the top
nn.fit(X_train, Y_train, epochs = epochs, batch_size=batch_size, verbose=verbose)

#also generate validation set by cutting the last validation_size elements from test data
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

#evaluates the score and the accuracy
score, accuracy = nn.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (accuracy))

Epoch 1/2
 - 55s - loss: 0.4244 - acc: 0.8173
Epoch 2/2
 - 53s - loss: 0.3386 - acc: 0.8565
score: 0.37
acc: 0.86


In [69]:
#initialize counter for evaluating prediction
pos_cnt, neg_cnt, pos_ok, neg_ok, tp, fp, tn ,fn = 0, 0, 0, 0, 0, 0, 0, 0 
for x in range(len(X_validate)):
    #predict for each element in validation set its true false probability
    result = nn.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    #print(result)
    #check if highest prob for same class
    if np.argmax(result) == np.argmax(Y_validate[x]):
        #if high prob in first array element ---> classification as neg ---> count neg 
        if np.argmax(Y_validate[x]) == 0: neg_ok += 1
        #else count as pos
        else: pos_ok += 1
    #count of teacher labels
    if np.argmax(Y_validate[x]) == 0: neg_cnt += 1
    else: pos_cnt += 1

#print results
print("pos_acc", pos_ok/pos_cnt*100, "%")
print("neg_acc", neg_ok/neg_cnt*100, "%")

pos_acc = pos_ok/pos_cnt
neg_acc =  neg_ok/neg_cnt

pos_acc 48.78048780487805 %
neg_acc 94.46540880503144 %


In [70]:
#evaluate on different dataset
X2 = ['Jonas is  nice and happy and in love and is looking for freedom']
X2 = tok.texts_to_sequences(X2)
X2 = pad_sequences(X2, maxlen=len(X[0]), dtype='int32', value=0)
print(X2)
print(nn.predict(X2, batch_size=1, verbose = 2)[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    5 700   8   8  12 218   8   5 290  14]]
[ 0.0937252  0.9062748]


In [78]:
configuration_1 = {
    "name":"experiment1",
    
    "model":"generic_model_1_name",
    "model-name":"deepNN",


    "dataset":"X1",
    "dataset-name":"dataset_sentiment.csv",
    "dataset-link":"see_github",
    
    "measurement-name":"generic_measurement_hub",
        
    "execution-name":"genereric_execution_name",
    
     "precision":-1,
    "recall":-1,    
    "pos_accn":pos_acc,
    "neg_acc":neg_acc,
    "accuracy":score
}

## Export Data

In [79]:
from rdflib import Namespace, Graph, Literal
from rdflib.namespace import FOAF, OWL, XSD, RDFS, DCTERMS, DOAP, DC, RDF

In [80]:
prov = Namespace('http://www.w3.org/ns/prov#')
dcat = Namespace('http://www.w3.org/ns/dcat#')
mexalgo = Namespace('http://mex.aksw.org/mex-algo#')
mexperf = Namespace('http://mex.aksw.org/mex-perf#')
mexcore = Namespace('http://mex.aksw.org/mex-core#')
this = Namespace('http://mex.aksw.org/examples/')

In [81]:
def experiment_root_graph(root_node_name):
    g = Graph()
    g.add((this[root_node_name],RDF.type, mexcore.Experiment))
    g.add((this[root_node_name],RDF.type, mexcore.ApplicationContext))
    g.add((this[root_node_name],RDFS.label, Literal('2719095')))
    g.add((this[root_node_name],DCTERMS.date, Literal('2018-05-28',datatype=XSD.date)))
    g.add((this[root_node_name],FOAF.givenName, Literal('Carsten')))
    g.add((this[root_node_name],FOAF.mbox, Literal('carsten.draschner@gmail.com')))
    return g ,this[root_node_name]



In [82]:
g, r = experiment_root_graph("first_test")
print(type(r))

<class 'rdflib.term.URIRef'>


In [83]:
#extend a rdf graph with a configuration
def experiment_to_rdf(graph, experiment, root_node):
    
    g.add((this[experiment["name"]],RDF.type,mexcore.ExperimentConfiguration))
    g.add((this[experiment["name"]],prov.used, this[experiment["model"]]))
    g.add((this[experiment["name"]],prov.wasStartedBy, root_node))
    
    #dataset information
    g.add((this[experiment["dataset"]],RDF.type,mexcore.Dataset))
    g.add((this.dataset2,RDFS.label,Literal(experiment["dataset-name"])))
    g.add((this.dataset2,DCTERMS.landingPage,Literal(experiment["dataset-link"])))
    
    #model description
    g.add((this[experiment["model"]],RDF.type,mexalgo.Algorithm))
    g.add((this[experiment["model"]],RDFS.label,Literal(experiment["model-name"])))
    g.add((this[experiment["model"]],DCTERMS.identifier,Literal(experiment["model-name"])))
    ###g.add((this.model1,mexalgo.hasHyperParameter,this.hyerparameter1))
    
    #execution
    g.add((this[experiment["execution-name"]],RDF.type,mexcore.ExecutionOverall))
    g.add((this[experiment["execution-name"]],prov.generated,this[experiment["measurement-name"]]))
    g.add((this[experiment["execution-name"]],prov.used,this.test))
    g.add((this[experiment["execution-name"]],prov.used,this[experiment["model"]]))
    
    #test
    g.add((this.test,RDF.type,mexcore.Test))
    g.add((this.test,RDFS.label,Literal('Test')))
    
    #evaluation information
    g.add((this[experiment["measurement-name"]],RDF.type,mexcore.PerformanceMeasure))
    g.add((this[experiment["measurement-name"]],mexperf.precision,Literal(experiment["precision"],datatype=XSD.float)))
    g.add((this[experiment["measurement-name"]],mexperf.recall,Literal(experiment["recall"],datatype=XSD.float)))
    g.add((this[experiment["measurement-name"]],mexperf.accuracy,Literal(experiment["accuracy"],datatype=XSD.float)))
    g.add((this[experiment["measurement-name"]],prov.wasGeneratedBy,this[experiment["execution-name"]]))

In [84]:
experiment_to_rdf(g,configuration_1,r)

In [85]:
with open('task3_metadata.ttl','wb') as f:
    f.write(g.serialize(format='turtle'))